In [1]:
import pdfminer
import requests
from bs4 import BeautifulSoup
from pdfminer import pdfparser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import string
from scipy.spatial.distance import cosine
import numpy as np
from nltk import PorterStemmer

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,\
                                  password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import  TextConverter # , XMLConverter, HTMLConverter
import urllib2
from urllib2 import Request
import datetime
import re

# Define a PDF parser function
def parsePDF(url):

    # Open the url provided as an argument to the function and read the content
    open = urllib2.urlopen(Request(url)).read()

    # Cast to StringIO object
    from StringIO import StringIO
    memory_file = StringIO(open)

    # Create a PDF parser object associated with the StringIO object
    parser = PDFParser(memory_file)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Define parameters to the PDF device objet 
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    codec = 'utf-8'

    # Create a PDF device object
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

    # Create a PDF interpreter object
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Process each page contained in the document
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data

In [4]:
def get_text(url):
    '''Function to pull all papers (pdf) off the arxiv'''
    base_url = url
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://arxiv.org'+ link for link in links]
    articles = []
    for url in urls:
        articles.append(parsePDF(url))
    return articles

In [5]:
#steph = convert_pdf_to_txt('/Users/Colin/Downloads/plms.pdf')
#physics1 = convert_pdf_to_txt('/Users/Colin/Downloads/physics1.pdf')
#shake = convert_pdf_to_txt('/Users/Colin/Downloads/shakespeare-tempest.pdf')
#steph2 = convert_pdf_to_txt('/Users/Colin/Downloads/twists_final.pdf')

In [6]:
%time
text= []
text = get_text('http://arxiv.org/list/math.NT/1508?show=150')


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [17]:
#Try to get rid of weird unicode characters
S = set()
S.update(letter for letter in string.lowercase)
S.update(letter for letter in string.uppercase)
S.update(digit for digit in string.digits)

In [18]:
new_text=[]
for i in range(len(text)):
    new_text.append([word for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])
    
for i in range(len(new_text)):
    new_text[i] = ' '.join(new_text[i])

In [19]:
#Make stop list
tfidf = TfidfVectorizer(stop_words='english')
Stop= set()
Stop.update([word for word in tfidf.get_stop_words()])
Stop.update(['theorem', 'lemma', 'proof', 'sum', 'difference', \
             'product', 'multiple', 'let', 'group', 'prime', 'log', 'limit', 'cid', 'result'\
            'main', 'conjecture', 'case', 'suppose', 'function', 'assume', 'follows', \
            'given', 'define', 'note', 'defined', 'class'])
Stop = list(Stop)

In [23]:

tfidf_math = TfidfVectorizer(max_features=100, stop_words=Stop, ngram_range=(1,1))
M = tfidf_math.fit_transform(new_text)

In [24]:
#Get some topics

feature_names = tfidf_math.get_feature_names()
nmf = NMF(n_components=10)
nmf.fit(M)
for topic_idx, topic in enumerate(nmf.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    print '\n'

Topic #0:
number real integer positive bound integers small primes zero constant


Topic #1:
abelian subgroup galois groups extensions extension image element algebraic elements


Topic #2:
series integral bound functions obtain formula fourier representation deﬁned using


Topic #3:
18 numbers type deﬁned number following integers formula functions theory


Topic #4:
modular forms adic quadratic fourier form module series free algebra


Topic #5:
sequence numbers deﬁned integer real following matrix values adic number


Topic #6:
lattice quadratic form points abelian space point polynomial basis small


Topic #7:
curve rational point degree polynomial points irreducible rank bound linear


Topic #8:
proposition non following denote local space exists corollary using main


Topic #9:
modulo primes coeﬃcients polynomial non section formula series number functions




In [26]:
#Get similarities
N = M.todense()
distances = []
for ix in range(N.shape[0]):
    distances.append(cosine(N[-2], N[ix]))
print np.argsort(distances)[0:10]
print np.around(sorted(distances)[0:10],4)

[104  89  13 102  88  77  67  63  42  16]
[ 0.      0.3479  0.3818  0.4207  0.4256  0.4703  0.4769  0.4986  0.5114
  0.52  ]


In [27]:
cosine(N[89], N[-2])

0.34789613433526301

In [28]:
dists = []
for ix in range(N.shape[0]):
    for jx in range(N.shape[0]):
        dists.append(cosine(N[ix], N[jx]))

In [31]:
dists = sorted(np.around(dists,4))

In [41]:
dists[106:126]

[0.1171,
 0.1171,
 0.12330000000000001,
 0.12330000000000001,
 0.12670000000000001,
 0.12670000000000001,
 0.20230000000000001,
 0.20230000000000001,
 0.2036,
 0.2036,
 0.20979999999999999,
 0.20979999999999999,
 0.22570000000000001,
 0.22570000000000001,
 0.26390000000000002,
 0.26390000000000002,
 0.2671,
 0.2671,
 0.27900000000000003,
 0.27900000000000003]